In [1]:
#!/usr/bin/env python3
# -*- coding:utf-8 -*-
import os
import json
import re
import csv
from scipy.stats import chi2_contingency
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix,f1_score,roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import math
np.random.seed(0)

/Users/linliu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
def feature_importance(dict_names):
    
    print("current dict: ",dict_names)

                
    def load_propaganda_task2():
        '''
        output:[[sentence, label]]
        '''
        global_sentence_label = list()

        tesk_2_3_path = '../other_dict/tasks-2-3/train/'
        target_data_2_dirs = os.listdir(tesk_2_3_path)
        article_name_set = set()
        for file_name in target_data_2_dirs:
            article_name_set.add(file_name.split(".")[0])
        for index in list(article_name_set):
            file_name = index + '.txt'
            text_name = tesk_2_3_path + file_name
            try:
                text = open(text_name, encoding='utf-8', mode='r')
            except:
                print('No such file or directory: ', text_name)
                continue
            label_name = tesk_2_3_path + index + '.task2.labels'
            try:
                label = open(label_name, encoding='utf-8', mode='r')
            except:
                print('No such file or directory: ', label_name)
                continue
            #print('text_name:',text_name[:2])
            text = text.readlines()
            #print('text:',text[:2])
            sentence_list = []
            for sentence in text:
                sentence_list.append(sentence)
            label = label.readlines()
            #print('label:',label[:2])
            label_list = []
            for line in label:
                label_list.append(line)
            assert len(label_list)==len(sentence_list),"text name %s, len of text %d, \
                    len of label %d"%(text_name, len(sentence_list), len(label_list))
            for index,sentence in enumerate(sentence_list):
                if "\tnon-propaganda" in label_list[index] and sentence.strip()!="":
                    global_sentence_label.append([sentence,0])
                elif "\tpropaganda" in label_list[index] and sentence.strip()!="":
                    global_sentence_label.append([sentence,1])
                else:
                    continue
        return global_sentence_label
    # load data
    
    global_sentence_label = load_propaganda_task2()
    #print('global_sentence_label:',global_sentence_label[:2])
    np.random.shuffle(global_sentence_label)
    # partition to positive/negative
    global_sentence_label_positive = [pair for pair in global_sentence_label if pair[1]==1]
    global_sentence_label_negative = [pair for pair in global_sentence_label if pair[1]==0][:3938]
#     print("num of sentence is %d"%(len(global_sentence_label)))
#     print("num of pos sentence is %d"%(len(global_sentence_label_positive)))
#     print("num of neg sentence is %d"%(len(global_sentence_label_negative)))
    # shuffle and partition to train/validation/test
    #np.random.shuffle(global_sentence_label_negative)
    #np.random.shuffle(global_sentence_label_positive)

    train_dataset = global_sentence_label_negative[:int(len(global_sentence_label_negative)*0.9)]
    train_dataset.extend(global_sentence_label_positive[:int(len(global_sentence_label_positive)*0.9)])

    validation_dataset = global_sentence_label_negative[int(len(global_sentence_label_negative)*0.9):]
    validation_dataset.extend(global_sentence_label_positive[int(len(global_sentence_label_negative)*0.9):])

    
    
    
    
    
    
    global_test_sentence_label = list()
    sentence_test_list = []
    label_test_list = []
    with open('../other_dict/new_propaganda_test/task2test_rs.csv','r') as csvfile_test_test:
        reader_test = csv.reader(csvfile_test_test)
        for i,rows in enumerate(reader_test):
            num_test=1
            if i in range(1,4032):
                #if i ==1:
                i==num_test
                row_test = rows
                #print('row_test:',row_test)
                #column = [row[1] for row in reader]
                text_name = row_test[:2]
                text_test = row_test[3]
                label =row_test[2]
                if label=="non-propaganda":
                    global_test_sentence_label.append([text_test,0])
                    #print('global_sentence_label_non:',global_sentence_label)
                elif label=="propaganda":
                    global_test_sentence_label.append([text_test,1])
                    #print('global_sentence_label:',global_sentence_label)
                else:
                    continue
     #print('global_test_sentence_label:',global_test_sentence_label[:2])

    # partition to positive/negative
    global_test_sentence_label_positive = [pair for pair in global_test_sentence_label if pair[1]==1]
    global_test_sentence_label_negative = [pair for pair in global_test_sentence_label if pair[1]==0]
    #print("num of sentence is %d"%(len(global_test_sentence_label)))
    #print("num of pos sentence is %d"%(len(global_test_sentence_label_positive)))
    #print("num of neg sentence is %d"%(len(global_test_sentence_label_negative)))
    # shuffle and partition to train/validation/test
    
    
    
    test_dataset = global_test_sentence_label_negative[:]
    test_dataset.extend(global_test_sentence_label_positive[:])
    #     print("num of train %d"%(len(train_dataset)))
    #     print("num of validation %d"%(len(validation_dataset)))
    #     print("num of test %d"%(len(test_dataset)))







    def load_dict(dict_path):
        word_set = set()
        dict_text = open(dict_path, encoding='utf-8', mode='r')
        for line in dict_text:
            if line.strip()!="":
                word =line.split(",")[0]
                word_set.add(word)
        return word_set
    def word_num_in_dict(sentence, word_set):
        try:
            word_list = sentence.strip().split(" ")
        except:
            print(sentence)
        word_num = 0
        for word in word_list:
            if word in word_set:
                word_num += 1
        return word_num
    def build_feature_matrix_and_label(dataset,dict_names,dict_path):
        num_sample = len(dataset)
        num_feature = len(dict_names)
        feature_matrix = np.zeros((num_sample, num_feature))
        label = np.zeros((num_sample,))
        for i in range(num_sample):
            for j in range(num_feature):
                if 'propaganda' in dict_names[j]:
                    dict_path='./result_fake_true_week5/'
                else:
                    dict_path="./result_fake_true_after_reduce_stopwords/"
                feature_matrix[i,j] = word_num_in_dict(dataset[i][0], load_dict(dict_path + dict_names[j] + "_words.csv"))
            label[i] = dataset[i][1]
        return feature_matrix,label
    def build_feature_matrix_new_one_dict(dataset,dict_name,dict_path):
        if 'propaganda' in dict_name:
            dict_path='./result_fake_true_week5/'
        else:
            dict_path="./result_fake_true_after_reduce_stopwords"
        word_list = list(load_dict(dict_path+dict_name+ "_words.csv"))
        num_sample = len(dataset)
        num_feature = len(word_list)
        feature_matrix = np.zeros((num_sample, num_feature))
        for i in range(len(dataset)):
            for j,word in enumerate(word_list):
                if word in dataset[i][0]:
                    feature_matrix[i][j]=1
        return(feature_matrix)
    #dict_names = ['fake_persuasive','fake_sentiment','fake_subjectivity','fake_technical','fake_all',
    #              'true_persuasive','true_sentiment','true_subjectivity','true_technical','true_all']
    #dict_names = ['fake_persuasive','fake_sentiment','fake_subjectivity','fake_technical',
    #              'true_persuasive','true_sentiment','true_subjectivity','true_technical']
    dict_path = "./result_fake_true_after_reduce_stopwords"
    train_feature_matrix, train_label = build_feature_matrix_and_label(train_dataset, dict_names, dict_path)
    validation_feature_matrix, validation_label = build_feature_matrix_and_label(validation_dataset, dict_names, dict_path)
    test_feature_matrix, test_label = build_feature_matrix_and_label(test_dataset, dict_names, dict_path)
    # one-hot feature
    for dict_name in dict_names:
        train_feature_matrix=np.hstack((build_feature_matrix_new_one_dict(train_dataset,dict_names[0],dict_path),train_feature_matrix))
        validation_feature_matrix=np.hstack((build_feature_matrix_new_one_dict(validation_dataset,dict_names[0],dict_path),validation_feature_matrix))
        test_feature_matrix=np.hstack((build_feature_matrix_new_one_dict(test_dataset,dict_names[0],dict_path),test_feature_matrix))

    #gbdt train again and predict
    train_feature_matrix = np.vstack((validation_feature_matrix,train_feature_matrix))
    
    
    train_label = np.concatenate((validation_label,train_label))
#     print(train_feature_matrix.shape, train_label.shape)
    gbdt = GradientBoostingClassifier(max_depth=4,
                                      random_state=0,
                                      min_samples_split=5,
                                      learning_rate=0.01,
                                      n_estimators=30,
                                      subsample=0.8)
    rf = gbdt.fit(train_feature_matrix, train_label)
    val_score_rbf = gbdt.score(train_feature_matrix, train_label)
#     print("The train accuracy score of rf is : %f" % val_score_rbf)
    #test_score_rbf = gbdt.score(test_feature_matrix, test_label)
    
    #print("Final: The test accuracy score of rf is : %f" % test_score_rbf )
    #predict_label = gbdt.predict(test_feature_matrix)
    
    prediction=gbdt.predict(test_feature_matrix)
    #test_score=f1_score(prediction,test_label)
    #print('f1:',f1_score(prediction,test_label))
    from sklearn.metrics import precision_score, recall_score, f1_score
    f1 = f1_score(prediction,test_label, average='binary')
    print("f1:", f1 )
    p = precision_score(prediction,test_label, average='binary')
    print("precision:", p)
    r = recall_score(prediction,test_label,average='binary')
    print("recall:", r)
    # confusion matrix 
    def plot_confusion_matrix(cm, title):
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]   
        plt.imshow(cm) 
        print(cm)
        plt.title(title)   
        plt.colorbar()
        plt.ylabel('True label')    
        plt.xlabel('Predicted label')
#     cm = confusion_matrix(test_label, predict_label)
#     print("confusion matrix is ", cm)
    #plot_confusion_matrix(cm, "RF Confusion Matrix")

In [5]:
all_dict = ['persuasive','sentiment','subjectivity','technical','all']
dict_num = len(all_dict)
f1score_list=[]
sub_list_all = []  
for i in range(1 << dict_num):  
    combo_list = [] 
    for j in range(dict_num):
        if i & (1 << j): 
            combo_list.append(all_dict[j]) 
    sub_list_all.append(combo_list)
#print(sub_list_all)
for ele in sub_list_all[1:]:
    dict_names = list()
    for i in ele:
        dict_names.extend(["fake_"+i, "true_"+i])
    feature_importance(dict_names)
feature_importance(['fake_subjectivity', 'true_subjectivity'])


print(sub_list_all)

current dict:  ['fake_persuasive', 'true_persuasive']
f1: 0.37396883593033914
precision: 0.3295638126009693
recall: 0.4322033898305085
current dict:  ['fake_sentiment', 'true_sentiment']
f1: 0.3527013251783894
precision: 0.27948303715670436
recall: 0.47790055248618785
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_sentiment', 'true_sentiment']
f1: 0.3672897196261682
precision: 0.3174474959612278
recall: 0.4356984478935698
current dict:  ['fake_subjectivity', 'true_subjectivity']
f1: 0.4382790510655408
precision: 0.44022617124394187
recall: 0.4363490792634107
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_subjectivity', 'true_subjectivity']
f1: 0.44488501189532126
precision: 0.4531502423263328
recall: 0.4369158878504673
current dict:  ['fake_sentiment', 'true_sentiment', 'fake_subjectivity', 'true_subjectivity']
f1: 0.40523465703971123
precision: 0.36268174474959614
recall: 0.4591002044989775
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_sentiment

In [46]:
from prettytable import PrettyTable
x= PrettyTable(["current dict", "f1"])
f1score_list=[]
all_dict = ['persuasive','sentiment','nrc_emotion','subjectivity','technical','all']
dict_num = len(all_dict)
sub_list_all = []  
for i in range(1 << dict_num):  
    combo_list = [] 
    for j in range(dict_num):
        if i & (1 << j): 
            combo_list.append(all_dict[j]) 
    sub_list_all.append(combo_list)
#print(sub_list_all)
for ele in sub_list_all[1:]:
    dict_names = list()
    for i in ele:
        dict_names.extend(["fake_"+i, "true_"+i])
    score=feature_importance(dict_names)
    x.add_row([ele,score])
                   
feature_importance(['fake_subjectivity', 'true_subjectivity'])
print(sub_list_all)
print(x)



current dict:  ['fake_persuasive', 'true_persuasive']
f1: 0.43225572103160187
current dict:  ['fake_sentiment', 'true_sentiment']
f1: 0.3664566165777994
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_sentiment', 'true_sentiment']
f1: 0.4444444444444444
current dict:  ['fake_nrc_emotion', 'true_nrc_emotion']
f1: 0.35525024533856725
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_nrc_emotion', 'true_nrc_emotion']
f1: 0.4570823030731191
current dict:  ['fake_sentiment', 'true_sentiment', 'fake_nrc_emotion', 'true_nrc_emotion']
f1: 0.41388066017774017
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_sentiment', 'true_sentiment', 'fake_nrc_emotion', 'true_nrc_emotion']
f1: 0.4131374243733794
current dict:  ['fake_subjectivity', 'true_subjectivity']
f1: 0.4510427712972782
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_subjectivity', 'true_subjectivity']
f1: 0.4713328369322412
current dict:  ['fake_sentiment', 'true_sentiment', 'fake_subjectivi

f1: 0.4855007473841555
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_sentiment', 'true_sentiment', 'fake_subjectivity', 'true_subjectivity', 'fake_technical', 'true_technical', 'fake_all', 'true_all']
f1: 0.48588885656095426
current dict:  ['fake_nrc_emotion', 'true_nrc_emotion', 'fake_subjectivity', 'true_subjectivity', 'fake_technical', 'true_technical', 'fake_all', 'true_all']
f1: 0.4835230677052127
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_nrc_emotion', 'true_nrc_emotion', 'fake_subjectivity', 'true_subjectivity', 'fake_technical', 'true_technical', 'fake_all', 'true_all']
f1: 0.4819860732667272
current dict:  ['fake_sentiment', 'true_sentiment', 'fake_nrc_emotion', 'true_nrc_emotion', 'fake_subjectivity', 'true_subjectivity', 'fake_technical', 'true_technical', 'fake_all', 'true_all']
f1: 0.46126760563380287
current dict:  ['fake_persuasive', 'true_persuasive', 'fake_sentiment', 'true_sentiment', 'fake_nrc_emotion', 'true_nrc_emotion', 'fake_subject